# Libraries

In [43]:
pip install nltk==3.5


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



     ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
      --------------------------------------- 0.0/1.4 MB 217.9 kB/s eta 0:00:07
      --------------------------------------- 0.0/1.4 MB 163.8 kB/s eta 0:00:09
      --------------------------------------- 0.0/1.4 MB 163.8 kB/s eta 0:00:09
     - -------------------------------------- 0.0/1.4 MB 151.3 kB/s eta 0:00:10
     - -------------------------------------- 0.0/1.4 MB 151.3 kB/s eta 0:00:10
     - -------------------------------------- 0.1/1.4 MB 164.1 kB/s eta 0:00:09
     -- ------------------------------------- 0.1/1.4 MB 208.4 kB/s eta 0:00:07
     -- ------------------------------------- 0.1/1.4 MB 218.5 kB/s eta 0:00:07
     --- ------------------------------------ 0.1/1.4 MB 242.7 kB/s eta 0:00:06
     --- ------------------------------------ 0.1/1.4 MB 225.3 kB/

In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from sklearn.model_selection import train_test_split
import wandb
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
import pandas as pd
import re
from transformers import MobileBertTokenizer, MobileBertForQuestionAnswering, AdamW
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from sklearn.model_selection import train_test_split
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu
from fractions import Fraction
import logging
from simpletransformers.question_answering import QuestionAnsweringModel



c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Data set 

In [2]:
df = pd.read_csv("train.csv")
df=df.head(100)

# Preprocessing 

In [3]:

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Cleaned Text

In [4]:

df['Question'] = df['Question'].apply(preprocess_text)
df['Answer'] = df['Answer'].apply(preprocess_text)

# Display the preprocessed data
df

,qtype,Question,Answer
0,susceptibility,who is at risk for lymphocytic choriomeningiti...,lcmv infections can occur after exposure to fr...
1,symptoms,what are the symptoms of lymphocytic choriomen...,lcmv is most commonly recognized as causing ne...
2,susceptibility,who is at risk for lymphocytic choriomeningiti...,individuals of all ages who come into contact ...
3,exams and tests,how to diagnose lymphocytic choriomeningitis lcm,during the first phase of the disease the most...
4,treatment,what are the treatments for lymphocytic chorio...,aseptic meningitis encephalitis or meningoence...
...,...,...,...
95,exams and tests,how to diagnose parasites leishmaniasis,various laboratory methods can be used to diag...
96,treatment,what are the treatments for parasites leishman...,before considering treatment the first step is...
97,prevention,how to prevent parasites leishmaniasis,no vaccines or drugs to prevent infection are ...
98,information,what is are parasites toxoplasmosis toxoplasma...,a singlecelled parasite called toxoplasma gond...


# Aplit the Data

In [5]:
train_data, test_data = train_test_split(df, test_size=0.25, random_state=42)

# PreProcess

In [6]:
train_examples = []
for _, row in train_data.iterrows():
    train_examples.append({
        'context': row['Answer'],  
        'qas': [{
            'id': row['qtype'], 
            'question': row['Question'],  
            'answers': [{
                'text': row['Answer'],  
                'answer_start': 0  
            }]
        }]
    })
test_examples = []
for _, row in test_data.iterrows():
    test_examples.append({
        'context': row['Answer'],  
        'qas': [{
            'id': str(row.name), 
            'question': row['Question'], 
            'answers': [{
                'text': row['Answer'],  
                'answer_start': 0  
            }]
        }]
    })

# Gineral Model ARguments

In [7]:

# Suppress unnecessary logging messages

# Define model arguments
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size = 5
model_args.num_train_epochs = 1
model_args.overwrite_output_dir = True
model_args.save_model_every_epoch = False
model_args.use_multiprocessing = False
model_args.wandb_project = "MedQuad-QA"
model_args.config = {
    "hidden_dropout_prob": 0.3,
    "attention_probs_dropout_prob": 0.3
}
model_args_7 = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size = 5  # Set n_best_size to 5
model_args.num_train_epochs = 1
model_args.overwrite_output_dir = True
model_args.save_model_every_epoch = False
model_args.use_multiprocessing = False
model_args.wandb_project = "MedQuad-QA"
model_args.config = {
    "hidden_dropout_prob": 0.7,  # Dropout rate set to 0.3
    "attention_probs_dropout_prob": 0.7  # Dropout rate set to 0.3
}
model_args.num_samples = 10  # Set the desired number of samples
logging.basicConfig(level=logging.ERROR)

# Load the Model  Bert

In [46]:
# Load the question answering model
bert_model_03_dropout = QuestionAnsweringModel(
    "bert", "bert-base-cased", args=model_args, use_cuda=torch.cuda.is_available()
)
bert_model_07_dropout = QuestionAnsweringModel(
    "bert", "bert-base-cased", args=model_args_7,use_cuda=torch.cuda.is_available()
)

c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# bert_model_03_dropout
5cb8eccab60f9a55795c159a6f083bd3089b708d

In [66]:
logging.basicConfig(level=logging.ERROR)
try:
    bert_model_03_dropout = QuestionAnsweringModel("bert", "bert-base-cased", args=model_args, use_cuda=torch.cuda.is_available())
    bert_model_03_dropout.train_model(train_examples, eval_data=test_examples, num_samples=5, output_dir="model_output/")
except Exception as e:
    print(f"An error occurred during training: {str(e)}")
try:
    bert_result, _ = bert_model_03_dropout.eval_model(test_examples, output_dir="model_output/", verbose=False)
    print("BERT Results:", bert_result)
    predictions = bert_model_03_dropout.predict(test_examples)
    references = [example['answers'][0]['text'] for example in test_examples]
except Exception as e:
    print(f"An error occurred during evaluation: {str(e)}")


c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1 of 1:   0%|          | 0/1 [00:10<?, ?it/s]c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\wandb\sdk\wandb_run.py:2265: UserWarning: Run (6ob2cuvl) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),
Epoch 1 of 1:   0%|          | 0/1 [03:23<?, ?it/s]


An error occurred during training: 'int' object is not callable


Running Evaluation: 100%|██████████| 1/1 [00:10<00:00, 10.88s/it]


BERT Results: {'correct': 0, 'similar': 25, 'incorrect': 0, 'eval_loss': -0.006273320410400629}


Running Prediction: 100%|██████████| 1/1 [00:10<00:00, 10.39s/it]

An error occurred during evaluation: 'answers'


# bert_model_07_dropout

In [85]:
logging.basicConfig(level=logging.ERROR)
try:
    bert_model_07_dropout = QuestionAnsweringModel("bert", "bert-base-cased", args=model_args_7, use_cuda=torch.cuda.is_available())
    bert_model_07_dropout.train_model(train_examples, eval_data=test_examples, num_samples=5, output_dir="model_output/")
except Exception as e:
    print(f"An error occurred during training: {str(e)}")
try:
    bert_result, _ = bert_model_07_dropout.eval_model(test_examples, output_dir="model_output/", verbose=False)
    print("BERT Results:", bert_result)
    predictions_07 = bert_model_07_dropout.predict(test_examples)
    extracted_predictions = []
    for pred in predictions:
        if 'answer' in pred:
            extracted_predictions.append(pred['answer'][0])
        else:
            extracted_predictions.append("")  # Appending an empty string if 'answer' key is not present

# Extracting reference answers
    references_07 = [example['context'] for example in test_examples]
except Exception as e:
    print(f"An error occurred during evaluation: {str(e)}")

# Blue Score 

In [88]:
min_len = min(len(predictions), len(references))
predictions = predictions[:min_len]
references = references[:min_len]
extracted_predictions = [pred['answer'][0] if isinstance(pred, dict) else pred for pred in predictions]
numerator = sum(min(len(pred), len(ref)) for pred, ref in zip(extracted_predictions, references))
denominator = sum(len(pred) for pred in extracted_predictions)
bleu_score = Fraction(numerator, denominator)
# for 07
min_len = min(len(predictions_07), len(references_07))
predictions_07 = predictions_07[:min_len]
references_07 = references_07[:min_len]
extracted_predictions = [pred['answer'][0] if isinstance(pred, dict) else pred for pred in predictions_07]
numerator_07 = sum(min(len(pred), len(ref)) for pred, ref in zip(extracted_predictions, references_07))
denominator_07 = sum(len(pred) for pred in extracted_predictions)
bleu_score_07 = Fraction(numerator_07, denominator_07)

print("BLEU Score:drop_out 03", bleu_score)
print("BLEU Score:drop_out 07", bleu_score_07)


BLEU Score:drop_out 03 0.6
BLEU Score:drop_out 07 0.8


# --------MObile Bert------------

In [18]:
model_args_mobilebert_03 = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "fp16": False,  # Set to True if you have FP16 training enabled
    "use_cuda": False,  # Set to False if CUDA is unavailable
    "num_train_epochs": 3,
    "dropout": 0.3,
}
model_args_mobilebert_07 = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "fp16": False,  # Set to True if you have FP16 training enabled
    "use_cuda": False,  # Set to False if CUDA is unavailable
    "num_train_epochs": 3,
    "dropout": 0.7,
}

In [20]:
# Train the MobileBERT model with dropout rate 0.3
try:
    mobilebert_model_03 = QuestionAnsweringModel("mobilebert", "google/mobilebert-uncased", args=model_args_mobilebert_03, use_cuda=torch.cuda.is_available())
    mobilebert_model_03.train_model(train_examples, eval_data=test_examples, num_samples=5, output_dir="model_output_mobilebert_03/")
except Exception as e:
    print(f"An error occurred during training with dropout rate 0.3: {str(e)}")

# Evaluate the MobileBERT model with dropout rate 0.3
try:
    mobilebert_result_03, _ = mobilebert_model_03.eval_model(test_examples)
    print("MobileBERT Results with dropout rate 0.3:", mobilebert_result_03)
    predictions_mobilebert_03 = mobilebert_model_03.predict(test_examples)
    references_mobilebert_03 = [example['context'] for example in test_examples]
except Exception as e:
    print(f"An error occurred during evaluation with dropout rate 0.3: {str(e)}")

Some weights of MobileBertForQuestionAnswering were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Running Evaluation: 100%|██████████| 1/1 [00:20<00:00, 20.09s/it]


MobileBERT Results with dropout rate 0.3: {'correct': 0, 'similar': 25, 'incorrect': 0, 'eval_loss': -70.57221984863281}


Running Prediction: 100%|██████████| 1/1 [00:05<00:00,  5.37s/it]


In [22]:
# Train the MobileBERT model with dropout rate 0.7
try:
    mobilebert_model_07 = QuestionAnsweringModel("mobilebert","google/mobilebert-uncased", args=model_args_mobilebert_07, use_cuda=torch.cuda.is_available())
    mobilebert_model_07.train_model(train_examples, eval_data=test_examples, num_samples=5, output_dir="model_output_mobilebert_07/")
except Exception as e:
    print(f"An error occurred during training with dropout rate 0.7: {str(e)}")

# Evaluate the MobileBERT model with dropout rate 0.7
try:
    mobilebert_result_07, _ = mobilebert_model_07.eval_model(test_examples)
    print("MobileBERT Results with dropout rate 0.7:", mobilebert_result_07)
    predictions_mobilebert_07 = mobilebert_model_07.predict(test_examples)
    references_mobilebert_07 = [example['context'] for example in test_examples]
except Exception as e:
    print(f"An error occurred during evaluation with dropout rate 0.7: {str(e)}")

Some weights of MobileBertForQuestionAnswering were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Running Evaluation: 100%|██████████| 1/1 [00:06<00:00,  6.67s/it]


MobileBERT Results with dropout rate 0.7: {'correct': 25, 'similar': 0, 'incorrect': 0, 'eval_loss': 20.118234634399414}


Running Prediction: 100%|██████████| 1/1 [00:04<00:00,  4.98s/it]


In [69]:
from nltk.translate.bleu_score import sentence_bleu

# Function to extract text from predictions
def extract_text(prediction):
    if isinstance(prediction, dict):
        return prediction['answer'][0]
    else:
        return prediction

# Compute BLEU score for dropout rate 0.3
min_len_03 = min(len(predictions_mobilebert_03), len(references_mobilebert_03))
predictions_03 = predictions_mobilebert_03[:min_len_03]
references_03 = [ref.split() for ref in references_mobilebert_03[:min_len_03]]  # Split each reference sentence into a list of words
extracted_predictions_03 = [extract_text(pred) for pred in predictions_03]
# Convert each extracted prediction into a single string
# Convert each extracted prediction into a single string
# Convert each extracted prediction into a single string
from fractions import Fraction

# Calculate numerator and denominator
numerator = sum(min(len(pred), len(ref)) for pred, ref in zip(extracted_predictions_03, references_03))
denominator = sum(len(pred) for pred in extracted_predictions_03)

# Manually normalize numerator and denominator
numerator = min(1, numerator)  # Ensure numerator is at least 1
denominator = max(1, denominator)  # Ensure denominator is at least 1

# Create Fraction object without _normalize argument
bleu_score_03 = Fraction(numerator, denominator)


min_len_07 = min(len(predictions_mobilebert_07), len(references_mobilebert_07))
predictions_07 = predictions_mobilebert_07[:min_len_03]
references_07= [ref.split() for ref in references_mobilebert_07[:min_len_03]]  # Split each reference sentence into a list of words
extracted_predictions_07 = [extract_text(pred) for pred in predictions_03]
numerator = sum(min(len(pred), len(ref)) for pred, ref in zip(extracted_predictions_07, references_07))
denominator = sum(len(pred) for pred in extracted_predictions_07)

# Manually normalize numerator and denominator
numerator = min(1, numerator)  
denominator = max(1, denominator)  

# Create Fraction object without _normalize argument
bleu_score_07 = Fraction(numerator, denominator)


print("BLEU Score for dropout rate 0.3:", bleu_score_03)
print("BLEU Score for dropout rate 0.7:", bleu_score_07)


BLEU Score for dropout rate 0.3: 1/50
BLEU Score for dropout rate 0.7: 1/50


# RoBERTa

In [8]:
# Initializing the RoBERTa Question Answering Model
roberta_model = QuestionAnsweringModel('roberta', 'roberta-base', use_cuda=torch.cuda.is_available())

# Training with limited data
print("Training RoBERTa model with limited data...")
roberta_model.train_model(train_examples, eval_data=test_examples)

# Evaluating the model
roberta_result, model_outputs = roberta_model.eval_model(test_examples)
print("RoBERTa Results with limited data:", roberta_result)

c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Error while downloading from https://cdn-lfs.huggingface.co/roberta-base/5bde1d28afb363d0103324efeb5afc8b2b397fe5e04beabb9b1ef355255ade81?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1716049936&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjA0OTkzNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yb2JlcnRhLWJhc2UvNWJkZTFkMjhhZmIzNjNkMDEwMzMyNGVmZWI1YWZjOGIyYjM5N2ZlNWUwNGJlYWJiOWIxZWYzNTUyNTVhZGU4MT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=A-5WhPyYJ6vxFBzujmMEvmxZGFD7Bq9wL9IDJ8PiswlFTUZMsIdSTIMQ7HqFuwSZI3C7nrDDYqMoZ35